data collected April 25, 2020 on a DigitalOcean droplet running Ubuntu 18.04

In [1]:
# the order of installation matters for these two :(
!pip install python-geoip-geolite2
!pip install python-geoip-python3

In [2]:
import pandas
print('pandas',pandas.__version__)
import re
import time
from geoip import geolite2
import sys
print(sys.version)
import matplotlib.pyplot as plt

pandas 0.23.4
3.6.6 | packaged by conda-forge | (default, Oct 12 2018, 14:08:43) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]


In [3]:
with open('gunicorn_access.log','r') as f:
    lines = f.readlines()

In [4]:
lines[0:5]

['172.18.0.3 - - [25/Apr/2020:21:25:50 +0000] "GET / HTTP/1.0" 200 17635 "-" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36"\n',
 '172.18.0.3 - - [25/Apr/2020:21:25:50 +0000] "GET /static/sorttable.js HTTP/1.0" 200 0 "http://167.172.250.37/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36"\n',
 '172.18.0.3 - - [25/Apr/2020:21:25:50 +0000] "GET /static/d3.min.js HTTP/1.0" 200 0 "http://167.172.250.37/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36"\n',
 '172.18.0.3 - - [25/Apr/2020:21:25:51 +0000] "GET /static/single_step.png HTTP/1.0" 200 0 "http://167.172.250.37/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36"\n',
 '172.18.0.3 - - [25/Apr/2020:21:25:51 +0000] "GET /static/mathjax/tex-s

In [5]:
list_of_dicts = []
for indx, line in enumerate(lines):
    line_as_dict = {}
    line_as_list = line.split(' ')
    line_as_dict['remote ip'] = line_as_list[0]
    line_as_dict['remote user'] = line_as_list[2]
    line_as_dict['date time'] = line_as_list[3]
    message = ' '.join(line_as_list[5:])
    # https://stackoverflow.com/questions/16710076/python-split-a-string-respect-and-preserve-quotes
    msg_as_list = re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', message)
    if len(msg_as_list)!=5:
        print(message)
    request = msg_as_list[0]
    request_as_list = request.split(' ')
    if (len(request_as_list)!=3):
        print('request does not match expected format:')
        print(line)
    else:
        line_as_dict['request method'] = request_as_list[0].replace('"','')
        line_as_dict['request file'] = request_as_list[1]
        line_as_dict['request proto'] = request_as_list[2].replace('"','')
    line_as_dict['status'] = msg_as_list[1]
    line_as_dict['body bytes sent'] = msg_as_list[2]
    line_as_dict['referrer'] = msg_as_list[3]
    line_as_dict['user agent'] = msg_as_list[4]
#    line_as_dict['gzip ratio'] = msg_as_list[5]

    list_of_dicts.append(line_as_dict)

In [6]:
df = pandas.DataFrame(list_of_dicts)
df.shape

(826, 10)

In [7]:
# https://strftime.org/
df['datetime'] = pandas.to_datetime(df['date time'], format='[%d/%b/%Y:%H:%M:%S')
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
df.drop('date time', inplace=True, axis=1)

In [8]:
df.head()

,body bytes sent,referrer,remote ip,remote user,request file,request method,request proto,status,user agent,datetime
0,17635,"""-""",172.18.0.3,-,/,GET,HTTP/1.0,200,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_...",2020-04-25 21:25:50
1,0,"""http://167.172.250.37/""",172.18.0.3,-,/static/sorttable.js,GET,HTTP/1.0,200,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_...",2020-04-25 21:25:50
2,0,"""http://167.172.250.37/""",172.18.0.3,-,/static/d3.min.js,GET,HTTP/1.0,200,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_...",2020-04-25 21:25:50
3,0,"""http://167.172.250.37/""",172.18.0.3,-,/static/single_step.png,GET,HTTP/1.0,200,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_...",2020-04-25 21:25:51
4,0,"""http://167.172.250.37/""",172.18.0.3,-,/static/mathjax/tex-svg.js,GET,HTTP/1.0,200,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_...",2020-04-25 21:25:51


In [9]:
df['remote ip'].value_counts()

172.18.0.3    826
Name: remote ip, dtype: int64

In [10]:
df['user agent'].value_counts()

"Mozilla/5.0 (X11; Linux x86_64; rv:68.0) Gecko/20100101 Firefox/68.0"                                                                           236
"Mozilla/5.0 (iPhone; CPU iPhone OS 13_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.5 Mobile/15E148 Safari/604.1"    143
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36"                            114
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)"                                                                                              92
"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0"                                                                    77
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36"                       51
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safa